# PAC Analysis

### Import libraries and read data

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import r2_score
from tensorpac import Pac

import seaborn as sns; sns.set_theme()

In [ ]:
path_pacs_dataset = "/path-to-pacs-dir/"
df = pd.read_parquet(path_pacs_dataset + "data_merged.parquet")
df.drop_duplicates(subset=['task', 'stimulus', 'medication', 'channel', 'subject', 'trial'], 
                    keep='first', inplace=True, ignore_index=True)
df = df.dropna(subset=["peak_time", "reaction_time"])
print(df.info())
print(df)

In [ ]:
pk_time = df["peak_time"].dropna().values
print("Average of Peak Time is: ", pk_time.mean())
print("STD of Peak Time is: ", pk_time.std())

### Plot PAC 2-Dimensional Distribution

In [ ]:
data_pac = df.iloc[0].pac_values
data_pac = data_pac.reshape(60, 30)
sns.heatmap(data_pac)

In [ ]:
pac = Pac(idpac=(5, 2, 0), f_pha=(0, 30.1, 1, 1), f_amp=(0, 60.1, 1, 1), dcomplex='wavelet')

#### Selected data for getting PAC images

* HCOFFMED
11 - hcoffmed - sham;
6154 - hcoffmed - stim7;
10110 - hcoffmed - stim8;

* PDOFFMED
17944 - pdoffmed - sham;
19838 - pdoffmed - stim7;
26315 - pdoffmed - stim8;

* PDONMED
29282 - pdonmed - sham;
34586 - pdonmed - stim7;
39684 - pdonmed - stim8;

### Analyzing Stimuli

In [ ]:
data = pd.read_parquet(path_pacs_dataset + "data_merged.parquet")
data.drop_duplicates(subset=['task', 'stimulus', 'medication', 'channel', 'subject', 'trial'], 
                    keep='first', inplace=True, ignore_index=True)
data = data.dropna(subset=["peak_time", "reaction_time"])
print(data)

In [ ]:
stim_df = pd.DataFrame([])
stim_df = data.groupby(['stimulus', 'subject'],
                        as_index=False)['peak_time'].mean()

rt_df = data.groupby(['stimulus', 'subject'],
                        as_index=False)['reaction_time'].mean()

mae_df = data.groupby(['stimulus', 'subject'],
                        as_index=False)['mae'].mean()

prd_df = data.groupby(['stimulus', 'subject'],
                        as_index=False)['predictions'].mean()

stim_df = pd.merge(stim_df, rt_df, on=['stimulus', 'subject'], how='outer')
stim_df = pd.merge(stim_df, mae_df, on=['stimulus', 'subject'], how='outer')
stim_df = pd.merge(stim_df, prd_df, on=['stimulus', 'subject'], how='outer')
stim_df.rename(columns={'peak_time': 'pt_avg', 'reaction_time': 'rt_avg',
                        'predictions': 'prd_avg', 'mae': 'mae_avg'},
                inplace=True)

In [ ]:
pt1 = stim_df[stim_df['stimulus'] == 'sham'].pt_avg
pt2 = stim_df[stim_df['stimulus'] == 'stim7'].pt_avg
pt3 = stim_df[stim_df['stimulus'] == 'stim8'].pt_avg

print(stats.ttest_ind(pt1, pt2))
print(stats.ttest_ind(pt1, pt3))
print(stats.ttest_ind(pt2, pt3))

In [ ]:
rt1 = stim_df[stim_df['stimulus'] == 'sham'].rt_avg
rt2 = stim_df[stim_df['stimulus'] == 'stim7'].rt_avg
rt3 = stim_df[stim_df['stimulus'] == 'stim8'].rt_avg

print(stats.ttest_ind(rt1, pt2))
print(stats.ttest_ind(rt1, pt3))
print(stats.ttest_ind(pt2, pt3))

In [ ]:
pac1 = np.concatenate(data[data['stimulus'] == 'sham'].pac_values.values, axis=0)
pac2 = np.concatenate(data[data['stimulus'] == 'stim7'].pac_values.values, axis=0)
pac3 = np.concatenate(data[data['stimulus'] == 'stim8'].pac_values.values, axis=0)

print(stats.ttest_ind(pac1, pac2))
print(stats.ttest_ind(pac1, pac3))
print(stats.ttest_ind(pac2, pac3))

In [ ]:
pac1 = data[data['stimulus'] == 'sham'].mae.values
pac2 = data[data['stimulus'] == 'stim7'].mae.values
pac3 = data[data['stimulus'] == 'stim8'].mae.values

print(stats.ttest_ind(pac1, pac2))
print(stats.ttest_ind(pac1, pac3))
print(stats.ttest_ind(pac2, pac3))

In [ ]:
mae1 = stim_df[stim_df['stimulus'] == 'sham'].mae_avg
mae2 = stim_df[stim_df['stimulus'] == 'sham'].mae_avg
mae3 = stim_df[stim_df['stimulus'] == 'sham'].mae_avg

print(stats.ttest_ind(mae1, mae2))
print(stats.ttest_ind(mae1, mae3))
print(stats.ttest_ind(mae2, mae3))

In [ ]:
df_stats = []
for stim in data.stimulus.unique():
    df_stim = data[data['stimulus'] == stim]
    for sbj in data.subject.unique():
        df_sbj = df_stim[df_stim['subject'] == sbj]
        if len(df_sbj) == 0:
            continue
        pred = df_sbj.predictions.values
        true = df_sbj.peak_time.values
        df_stats.append({'stimulus': stim,
                        'subject': sbj,
                        'corr': r2_score(true, pred)
                        }
                        )

df_ss = pd.DataFrame(df_stats)

In [ ]:
mc1 = df_ss[df_ss['stimulus'] == 'sham']['corr'].values
mc2 = df_ss[df_ss['stimulus'] == 'stim7']['corr'].values
mc3 = df_ss[df_ss['stimulus'] == 'stim8']['corr'].values

print(stats.ttest_ind(mc1, mc2))
print(stats.ttest_ind(mc1, mc3))
print(stats.ttest_ind(mc2, mc3))

In [ ]:
# Initialize an empty DataFrame to store results
result_df = pd.DataFrame(columns=['Pair', 'Statistic', 'P-value'])
for col in data.columns:
    statistic, p_value = stats.kruskal(*[group[col] for label, group in data.groupby("stimulus")])
    result_df = result_df.append({'Pair': f'{col} vs Label', 'Statistic': statistic, 'P-value': p_value}, ignore_index=True)

print(result_df)

### Analyzing Medication

In [ ]:
data = pd.read_parquet(path_pacs_dataset + "data_merged.parquet")
data.drop_duplicates(subset=['task', 'stimulus', 'medication', 'channel', 'subject', 'trial'], 
                    keep='first', inplace=True, ignore_index=True)
data = data.dropna(subset=["peak_time", "reaction_time"])
print(data)

In [ ]:
print(data['medication'].unique())
data['medication'].replace('pdoffmed_gvs8', 'pdoffmed', inplace=True)
data['medication'].replace('hcoffmed_gvs8', 'hcoffmed', inplace=True)
data['medication'].replace('pdonmed_gvs8', 'pdonmed', inplace=True)
print(data['medication'].unique())

In [ ]:
stim_df = pd.DataFrame([])
stim_df = data.groupby(['medication', 'subject'], as_index=False)['peak_time'].mean()

rt_df = data.groupby(['medication', 'subject'], as_index=False)['reaction_time'].mean()

mae_df = data.groupby(['medication', 'subject'], as_index=False)['mae'].mean()

prd_df = data.groupby(['medication', 'subject'], as_index=False)['predictions'].mean()

stim_df = pd.merge(stim_df, rt_df, on=['medication', 'subject'], how='outer')
stim_df = pd.merge(stim_df, mae_df, on=['medication', 'subject'], how='outer')
stim_df = pd.merge(stim_df, prd_df, on=['medication', 'subject'], how='outer')
stim_df.rename(columns={'peak_time': 'pt_avg', 'reaction_time': 'rt_avg',
                        'predictions': 'prd_avg', 'mae': 'mae_avg'},
                inplace=True)

In [ ]:
pt1 = stim_df[stim_df['medication'] == 'hcoffmed'].pt_avg
pt2 = stim_df[stim_df['medication'] == 'pdoffmed'].pt_avg
pt3 = stim_df[stim_df['medication'] == 'pdonmed'].pt_avg

print(stats.ttest_ind(pt1, pt2))
print(stats.ttest_ind(pt1, pt3))
print(stats.ttest_ind(pt2, pt3))

In [ ]:
rt1 = stim_df[stim_df['medication'] == 'hcoffmed'].rt_avg
rt2 = stim_df[stim_df['medication'] == 'pdoffmed'].rt_avg
rt3 = stim_df[stim_df['medication'] == 'pdonmed'].rt_avg

print(stats.ttest_ind(rt1, pt2))
print(stats.ttest_ind(rt1, pt3))
print(stats.ttest_ind(pt2, pt3))

In [ ]:
pac1 = np.concatenate(data[data['medication'] == 'hcoffmed'].pac_values.values,
                      axis=0)
pac2 = np.concatenate(data[data['medication'] == 'pdoffmed'].pac_values.values,
                      axis=0)
pac3 = np.concatenate(data[data['medication'] == 'pdonmed'].pac_values.values,
                      axis=0)

print(stats.ttest_ind(pac1, pac2))
print(stats.ttest_ind(pac1, pac3))
print(stats.ttest_ind(pac2, pac3))

In [ ]:
mae1 = stim_df[stim_df['medication'] == 'hcoffmed'].mae_avg
mae2 = stim_df[stim_df['medication'] == 'pdoffmed'].mae_avg
mae3 = stim_df[stim_df['medication'] == 'pdonmed'].mae_avg

print(stats.ttest_ind(mae1, mae2))
print(stats.ttest_ind(mae1, mae3))
print(stats.ttest_ind(mae2, mae3))

In [ ]:
df_ms = []
for med in data.medication.unique():
    df_med = data[data['medication'] == med]
    for sbj in data.subject.unique():
        df_sbj = df_med[df_med['subject'] == sbj]
        if len(df_sbj) == 0:
            continue
        pred = df_sbj.predictions.values
        true = df_sbj.peak_time.values
        df_ms.append({'medication': med,
                      'subject': sbj,
                      'corr': r2_score(true, pred)
                      },
                     )

df_ms = pd.DataFrame(df_ms)

In [ ]:
mc1 = df_ms[df_ms['medication'] == 'hcoffmed']['corr'].values
mc2 = df_ms[df_ms['medication'] == 'pdoffmed']['corr'].values
mc3 = df_ms[df_ms['medication'] == 'pdonmed']['corr'].values

print(stats.ttest_ind(mc1, mc2))
print(stats.ttest_ind(mc1, mc3))
print(stats.ttest_ind(mc2, mc3))

### Evaluating Correlation

### EA Data

In [ ]:
print(data['medication'].unique())
data['medication'].replace('pdoffmed_gvs8', 'pdoffmed', inplace=True)
data['medication'].replace('hcoffmed_gvs8', 'hcoffmed', inplace=True)
data['medication'].replace('pdonmed_gvs8', 'pdonmed', inplace=True)
print(data['medication'].unique())

In [ ]:
stim_df = pd.DataFrame([])
stim_df = data.groupby(['stimulus', 'medication', 'subject'], as_index=False)['peak_time'].mean()

rt_df = data.groupby(['stimulus', 'medication', 'subject'], as_index=False)['reaction_time'].mean()

mae_df = data.groupby(['stimulus', 'medication', 'subject'], as_index=False)['mae'].mean()

prd_df = data.groupby(['stimulus', 'medication', 'subject'], as_index=False)['predictions'].mean()

stim_df = pd.merge(stim_df, rt_df, on=['stimulus', 'medication', 'subject'], how='outer')
stim_df = pd.merge(stim_df, mae_df, on=['stimulus', 'medication', 'subject'], how='outer')
stim_df = pd.merge(stim_df, prd_df, on=['stimulus', 'medication', 'subject'], how='outer')
stim_df.rename(columns={'peak_time': 'pt_avg', 'reaction_time': 'rt_avg',
                        'predictions': 'prd_avg', 'mae': 'mae_avg'},
                inplace=True)

In [ ]:
path_results_analyzing = "/path-to-analyzing-results/"
stim_df.to_csv(path_results_analyzing + "ea_data.csv", index=False)

In [ ]:
df = pd.read_csv(path_results_analyzing + 'ea_data.csv')
print(df.info())
print(df)

### Plot input data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.io import loadmat

In [ ]:
path_to_eeg_data = "/path-to-eeg-data/"
data = loadmat(path_to_eeg_data + "PT_TimeLocked_1000Back.MAT")

In [ ]:
stim, subject = 1, 8
channel, trial = 7, 6

data_task = data["dataHCTask2"]
eeg_data = data_task[stim, subject]
signal = eeg_data[channel-1, :, trial-1]

In [ ]:
plt.plot(signal)
plt.xlabel('Time (ms)')
plt.ylabel('Value')
plt.title('EEG data')
plt.grid()
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 4), tight_layout=True)

sns.set_style("white", {'font.family':'serif', 'font.serif':'Times New Roman'})

ax = sns.lineplot(data=signal, color='blue', linewidth=2)

# ax.set(title='Histogram', xlabel='Peak Time (milliseconds)', ylabel='Count')
ax.axes.set_title("EEG signal", fontsize=15)
ax.set_xlabel("Time (milliseconds)", fontsize=12)
ax.set_ylabel("Value", fontsize=12)
ax.set_xlim(0, 1000)
ax.set_ylim(-30, 30)
# ax.tick_params(labelsize=10)
# ax.legend(["Count"])
fig.savefig('eeg_dist.png')

In [ ]:
pk_time = df["peak_time"].values
bins=50

fig = plt.figure(figsize=(6, 4), tight_layout=True)

# plt.hist(pk_time, bins=bins, color=sns.color_palette('Set2')[2], linewidth=2)

sns.set_style("white", {'font.family':'serif', 'font.serif':'Times New Roman'})

ax = sns.histplot(data=pk_time, bins=bins, color='blue', linewidth=2)

# ax.set(title='Histogram', xlabel='Peak Time (milliseconds)', ylabel='Count')
ax.axes.set_title("Histogram", fontsize=15)
ax.set_xlabel("Peak Time (milliseconds)", fontsize=12)
ax.set_ylabel("Count", fontsize=12)
# ax.tick_params(labelsize=10)
# ax.legend(["Count"])
fig.savefig('pt_dist.png')

In [ ]:
import matplotlib as mpl

font_paths = mpl.font_manager.findSystemFonts()
font_objects = mpl.font_manager.createFontList(font_paths)
font_names = [f.name for f in font_objects]

print(font_names)

### Representational Similarity Analysis

In [ ]:
df_temp = df.pac_values
df_pacs = pd.DataFrame(np.vstack(df_temp)).T

df_pacs.corr()

In [ ]:
print(df['medication'].unique())
df['medication'].replace('pdoffmed_gvs8', 'pdoffmed', inplace=True)
df['medication'].replace('hcoffmed_gvs8', 'hcoffmed', inplace=True)
df['medication'].replace('pdonmed_gvs8', 'pdonmed', inplace=True)
df["ID"] = df.index
print(df['medication'].unique())

In [ ]:
from scipy.spatial import distance

rsa_dir = "/content/drive/MyDrive/Projects/UBC_PI/generated_data/Statistics/"
 
# df_sampled = df.sample(n=int(2e3)).reset_index()
df_sampled = df
rsa = []
for index, row in df_sampled.iterrows():
    df_not_row = df_sampled[df_sampled["ID"] != index]
    for idx, item in df_not_row.iterrows():
        rsa.append({
                "stimulus_in": row['stimulus'],
                "stimulus_cmp": item["stimulus"],
                "med_in": row["medication"],
                "med_cmp": item["medication"],
                "subject_in": row["subject"],
                "subject_cmp": item["subject"],
                "r2_score": r2_score(row["pac_values"], item["pac_values"]),
                "cosine": np.dot(row["pac_values"], item["pac_values"])\
                /(np.linalg.norm(row["pac_values"])*np.linalg.norm(item["pac_values"])),
                "corrcoef": np.corrcoef(row["pac_values"], item["pac_values"])[0,1],
                "js": distance.jensenshannon(row["pac_values"], item["pac_values"])
        })
        # print(f"Compared iteration is {idx}")
    print()
    print(f"In iteration is {index}")

rsa = pd.DataFrame(rsa)
rsa.to_csv(rsa_dir + "rsa.csv", index=False)

In [ ]:
import pandas as pd

rsa_dir = "/content/drive/MyDrive/Projects/UBC_PI/generated_data/Statistics/"
rsa = pd.read_csv(rsa_dir + "rsa.csv")

In [ ]:
metrics = ["r2_score", "cosine", "corrcoef", "js"]
df_stim = rsa.groupby(["stimulus_in", "stimulus_cmp"])[metrics[3]].max()
print(df_stim)
print()

df_med = rsa.groupby(["med_in", "med_cmp"])[metrics[3]].max()
print(df_med)
print()

df_case = rsa.groupby(["subject_in", "subject_cmp"])[metrics[3]].max()
print(df_case)

### Barplot of ROIs

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
path_results = "/path-to-results-analyzing/"
data = pd.read_csv(path_results + "regions_stats.csv")

In [ ]:
rois_order = ['Delta_Beta', 'Theta_Beta', 'Beta_Gamma', 'Delta_Theta',
            'Theta_Gamma', 'Delta_Gamma', 'Theta_Alpha',
            'Alpha_Beta', 'Alpha_Gamma', 'Delta_Alpha', 'Overall']

In [ ]:
hc = data[data["case"] == "hc"]
pdon = data[data["case"] == "pdon"]
pdoff = data[data["case"] == "pdoff"]

In [ ]:
# The values are their corresponding average!
fig = plt.figure(figsize=(14, 6), tight_layout=True)
graph = sns.barplot(data=hc, x='Region', y='mean', hue='stimulus', order=rois_order,
                    palette=["darkslateblue", "orangered", "forestgreen"])
graph.axhline(0.238952, ls='--', c='blue') #stim7
graph.axhline(0.252774, ls='--', c='red') #stim8
graph.axhline(0.249524, ls='--', c='green') #sham

graph.axes.set_title("Regions Of Interest Importance For HC", fontsize=17)
graph.set_xlabel("Region of Interest", fontsize=12)
graph.set_ylabel("Mean", fontsize=12)

In [ ]:
fig = plt.figure(figsize=(14, 6), tight_layout=True)
graph = sns.barplot(data=pdon, x='Region', y='mean', hue='stimulus', order=rois_order,
                    palette=["darkslateblue", "orangered", "forestgreen"])
graph.axhline(0.259467, ls='--', c='red') #stim7
graph.axhline(0.287867, ls='--', c='blue') #stim8
graph.axhline(0.245912, ls='--', c='green') #sham

graph.axes.set_title("Regions Of Interest Importance For PD-On", fontsize=17)
graph.set_xlabel("Region of Interest", fontsize=12)
graph.set_ylabel("Mean", fontsize=12)

In [ ]:
fig = plt.figure(figsize=(14, 6), tight_layout=True)
graph = sns.barplot(data=pdoff, x='Region', y='mean', hue='stimulus',
                 order=rois_order, palette=["darkslateblue", "orangered", "forestgreen"])
graph.axhline(0.247691, ls='--', c='red') #sham
graph.axhline(0.254075, ls='--', c='green') #stim7
graph.axhline(0.237493, ls='--', c='blue') #stim8

graph.axes.set_title("Regions Of Interest Importance For PD-Off", fontsize=17)
graph.set_xlabel("Region of Interest", fontsize=12)
graph.set_ylabel("Mean", fontsize=12)